<a href="https://colab.research.google.com/github/SayanthT/CTC-Prediction/blob/main/leaf_decease_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#import Keras packages
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np

In [5]:
# Initializing the CNN

np.random.seed(1337)
classifier = Sequential()

classifier.add(Convolution2D(32, 3, 3, input_shape = (128, 128, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(16, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#flattening
classifier.add(Flatten())

#hidden layer
classifier.add(Dense(128, activation = 'relu'))


#output layer
classifier.add(Dense(10, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(classifier.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 42, 42, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 21, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 16)          4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 144)               0         
                                                                 
 dense (Dense)               (None, 128)               1

In [6]:
# fitting the data set

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/tomato/train',
        target_size=(128, 128),
        batch_size=64,
        class_mode='categorical' )
label_map = (training_set.class_indices)

print(label_map)

test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/tomato/val',
        target_size=(128, 128),
        batch_size=64,
        class_mode='categorical')

Found 10000 images belonging to 10 classes.
{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites Two-spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}
Found 1000 images belonging to 10 classes.


In [7]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=20,
        epochs=100,
        validation_data=test_set,
        validation_steps=100)


Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 2.3038 - accuracy: 0.1180 

20/20 [==============================] - 1673s 85s/step - loss: 2.3038 - accuracy: 0.1180 - val_loss: 2.2915 - val_accuracy: 0.1000
Epoch 2/100
20/20 [==============================] - 809s 41s/step - loss: 2.2761 - accuracy: 0.1156
Epoch 3/100
20/20 [==============================] - 624s 31s/step - loss: 2.1859 - accuracy: 0.1891
Epoch 4/100
20/20 [==============================] - 593s 30s/step - loss: 2.0487 - accuracy: 0.3016
Epoch 5/100
20/20 [==============================] - 454s 23s/step - loss: 1.9537 - accuracy: 0.3203
Epoch 6/100
20/20 [==============================] - 428s 21s/step - loss: 1.8989 - accuracy: 0.3352
Epoch 7/100
20/20 [==============================] - 356s 18s/step - loss: 1.7609 - accuracy: 0.4026
Epoch 8/100
20/20 [==============================] - 292s 15s/step - loss: 1.6658 - accuracy: 0.4234
Epoch 9/100
20/20 [==============================] - 268s 14s/step - loss: 1.5516 - accuracy: 0.4797
Epoch 10/100
20/20 [==============================] - 238s 1

In [8]:
classifier.evaluate_generator(test_set)

[0.5794369578361511, 0.8169999718666077]

In [9]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
path='/content/drive/MyDrive/tomato/train/Tomato___Bacterial_spot/00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG'
img=imread(path)
img=resize(img,(1,128,128,3))
res=classifier.predict_generator(img)
print(res)
decease=['Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite',
         'Tomato___Target_Spot','Tomato___Tomato_Yellow_Leaf_Curl_Virus','Tomato___Tomato_mosaic_virus','Tomato___healthy']
print('given image belongs to',decease[np.argmax(res)])

[[4.5856607e-01 2.0073095e-01 3.0283034e-01 9.1124204e-04 2.2572507e-04
  3.6823156e-03 8.8795619e-03 1.2326604e-04 2.6613239e-07 2.4050310e-02]]
given image belongs to Tomato___Bacterial_spot


**Leaf decease names and there corresponding values**

Tomato___Bacterial_spot  ----->   0

Tomato___Early_blight    ----->   1

Tomato___Late_blight     ----->   2

Tomato___Leaf_Mold       ----->   3

Tomato___Septoria_leaf_spot    ----->   4

Tomato___Spider_mites Two-spotted_spider_mite   ----->   5

Tomato___Target_Spot   ----->   6

Tomato___Tomato_Yellow_Leaf_Curl_Virus  ----->  7

Tomato___Tomato_mosaic_virus   ----->  8

Tomato___healthy    -----> 9